### Detach one from another

In [1]:
import trimesh
import numpy as np
from utils import *

In [2]:
mesh = trimesh.load("data/two_spheres2.ply")

Split the mesh into connected components

In [3]:
connected_components = mesh.split(only_watertight=True)

Compute centroids of each component

In [4]:
centroids = [comp.centroid for comp in connected_components]

Compute the vector between the centroids

In [5]:
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance

Shift the second component along the normalized vector

In [ ]:
connected_components[1].apply_translation(normalized_vector * 0.7)

<trimesh.Trimesh(vertices.shape=(476, 3), faces.shape=(948, 3), name=`two_spheres2.ply`)>

In [7]:
new_mesh = trimesh.util.concatenate(connected_components)

In [8]:
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)

In [9]:
visualize_two_meshes(mesh, x)

Widget(value='<iframe src="http://localhost:37609/index.html?ui=P_0x7f3b9e3a10f0_0&reconnect=auto" class="pyvi…

None

## Detach mesh

In [2]:
import trimesh
import numpy as np
from utils import *

In [ ]:
### Generating a sample mesh
"""
mesh = trimesh.load("data/shrink.ply")
connected_components = mesh.split(only_watertight=True)
centroids = [comp.centroid for comp in connected_components]
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance
connected_components[1].apply_translation(normalized_vector * 0.079)
new_mesh = trimesh.util.concatenate(connected_components)
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)
print(pymesh.detect_self_intersection(x))
pymesh.save_mesh("data/shrink.ply", x)
"""

[[ 465  844]
 [ 465 1199]
 [ 465  830]
 [ 465 1156]
 [ 828  465]
 [1154  465]
 [1211  465]
 [ 864  465]
 [1232  465]
 [ 825  465]
 [1194  465]]


In [67]:
import trimesh
import open3d as o3d
import numpy as np
from utils import *
from utils import *

In [68]:
def trimesh_to_open3d(tri_mesh: trimesh.Trimesh) -> o3d.geometry.TriangleMesh:
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(tri_mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(tri_mesh.faces)
    return o3d_mesh

def pymesh_to_trimesh(mesh):
    return trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)

def trimesh_to_pymesh(mesh):
    return pymesh.form_mesh(vertices=mesh.vertices, faces=mesh.faces)

def open3d_to_pymesh(o3d_mesh: o3d.geometry.TriangleMesh) -> pymesh.Mesh:
    vertices = np.asarray(o3d_mesh.vertices)
    faces = np.asarray(o3d_mesh.triangles)
    return  pymesh.form_mesh(vertices, faces)

In [69]:
def detach_inner_component(
    input_path,
    contact_threshold=0.001,
    num_sample_points=50000
):
    """
    1) Use trimesh to read and split (connected components).
    2) Convert each submesh to Open3D geometry.
    3) Push the inner mesh inward where it contacts the outer mesh.
    """
    # 1) Read mesh with trimesh and split into multiple components
    tm = trimesh.load(input_path, process=False)
    submeshes = tm.split(only_watertight=True) 
    
    # 2) Sort by volume
    submeshes_sorted = sorted(submeshes, key=lambda m: m.volume)
    mesh_inner_tm = submeshes_sorted[0]
    mesh_outer_tm = submeshes_sorted[-1]
    
    # 3) Convert to Open3D
    mesh_inner = trimesh_to_open3d(mesh_inner_tm)
    mesh_outer = trimesh_to_open3d(mesh_outer_tm)
    
    # For visualization and processing later
    mesh_inner.compute_vertex_normals()
    mesh_outer.compute_vertex_normals()

    # Sample outer for distance checks
    # Generate num_sample_points uniformly distributed points on the surface of mesh_outer. This helps in efficiently computing the nearest neighbors.
    pcd_outer = mesh_outer.sample_points_poisson_disk(number_of_points=num_sample_points)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_outer)

    inner_vertices = np.asarray(mesh_inner.vertices)
    center_inner = np.mean(inner_vertices, axis=0)

    # For each vertex, find distance to outer sphere
    for i in range(len(inner_vertices)):
        v = inner_vertices[i]
        # k: # of nearest neighors found (should be 1)
        # idx: Index of the nearest neighbor (outersurface)
        # dist_sq: Squared distance to the nearst neighbor
        k, idx, dist_sq = pcd_tree.search_knn_vector_3d(v, 1) # 1 -> We want a nearest neighbor
        if k > 0:
            closest_point = np.asarray(pcd_outer.points)[idx[0]]
            dist = np.linalg.norm(v - closest_point)
            print(dist)
            if dist < contact_threshold:
                # shrink inward by (contact_threshold - dist)
                offset = contact_threshold - dist
                direction = v - center_inner
                r_len = np.linalg.norm(direction)
                if r_len > 1e-12:
                    direction_unit = direction / r_len
                    inner_vertices[i] = v - offset * direction_unit


    mesh_inner.vertices = o3d.utility.Vector3dVector(inner_vertices)

    # Merge the two meshes
    combined_mesh = mesh_inner + mesh_outer
    return combined_mesh


input_ply = "data/shrink.ply"
final = detach_inner_component(input_path=input_ply, contact_threshold=0.03, num_sample_points=50000)

0.15174382028984734
0.10271580593512712
0.12595192967977561
0.1094000787329456
0.06338265389169823
0.08543822385286422
0.09304845251363987
0.04139058208516351
0.0657615088432917
0.11416712315027365
0.08833317312826063
0.039019851598980096
0.05910480225004458
0.09875560125558888
0.11422688028492113
0.07263165275142384
0.05060660772500468
0.043659713150315554
0.023202013056616674
0.029827584034184455
0.03584581449420547
0.010068533758231768
0.024565645337145395
0.0379043478949616
0.050716393365431045
0.02681887257824601
0.017740948919064362
0.0661986061427297
0.03229445737265595
0.06410942698400957
0.010233168337244015
0.003943348559322186
0.005044074046464657
0.005670889600103792
0.02967198795409586
0.009580251900758473
0.06655350605190882
0.035714408557263835
0.014206715072995623
0.002471005070017933
0.019897598483884376
0.004069864929523681
0.005295691202745829
0.013331220104253907
0.021435954000089828
0.03419161476021514
0.1675961857284172
0.20110297483757705
0.20431212833944726
0.24

In [71]:
original = pymesh.load_mesh("data/shrink.ply")
final = open3d_to_pymesh(final)
print(pymesh.detect_self_intersection(original))
print(pymesh.detect_self_intersection(final))

[[ 465  844]
 [ 465 1199]
 [ 465  830]
 [ 465 1156]
 [ 828  465]
 [1154  465]
 [1211  465]
 [ 864  465]
 [1232  465]
 [ 825  465]
 [1194  465]]
[]


In [72]:
visualize_two_meshes(original, final)

Widget(value='<iframe src="http://localhost:41219/index.html?ui=P_0x7f6858377400_9&reconnect=auto" class="pyvi…

None

In [73]:
tri_original = pymesh_to_trimesh(original)
tri_final = pymesh_to_trimesh(final)

In [74]:
original_submeshes = tri_original.split(only_watertight=True)
original_inner = sorted(original_submeshes, key=lambda m: m.volume)[0]
final_submeshes = tri_final.split(only_watertight=True)
final_inner = sorted(final_submeshes, key=lambda m: m.volume)[0]

In [75]:
evaluation(original_inner, final_inner)

+-----------------------------------+-----------+-----------+
| Metric                            |    Before |     After |
+===================================+===========+===========+
| Number of vertices                | 368       | 368       |
+-----------------------------------+-----------+-----------+
| Number of faces                   | 732       | 732       |
+-----------------------------------+-----------+-----------+
| Number of intersecting face pairs |   0       |   0       |
+-----------------------------------+-----------+-----------+
| Volume                            |   1.27715 |   1.27028 |
+-----------------------------------+-----------+-----------+
| Area                              |   5.7148  |   5.69626 |
+-----------------------------------+-----------+-----------+
| Intact vertices (%)               | nan       |  91.5761  |
+-----------------------------------+-----------+-----------+
